In [ ]:
from math import sqrt
import numpy as np
import shared_tools as tools
import GSD_calculator as gsd_calc
reload(gsd_calc)

import matplotlib.pyplot as plt
%matplotlib

class AgDegNormalGravMixPW(object):
    
    def __init__(self,
                 friction = 'Manning',
                 Q_flood = 100,
                 Q_gravel = 0.01,
                 channel_width = 10,
                 intermittency = 0.1,
                 base_level_elevation = 0.5,
                 reach_length = 20000,
                 dx = 666,
                 bed_slope = 0.002,
                 porosity = 0.4,
                 roughness_factor = 2,
                 active_layer_factor = 2,
                 manning_coeff = 8.1,
                 substrate_coeff = 0.5,
                 chezy_friction_coeff = 11.099,
                 upwind_coeff = 0.5,
                 dt = 0.01,
                 totalT = 10,
                 g = 9.81,
                 rho_w = 1000,
                 rho_s = 2650,
                 grain_diameters = None):
    
    
        # grain size distributions - list of tuples of diameter, list of GSD of feed, surface, substrate
        GSD = [(256,[100,100,100]),
               (128,[95,95,95]),
               (64,[80,80,80]),
               (32,[50,50,50]),
               (16,[25,25,25]),
               (8,[10,10,10]),
               (4,[5,5,5]),
               (2,[0,0,0]),
               (1,[0,0,0]),
               (0.5,[0,0,0]),
               (0.25,[0,0,0]),
               (0.125,[0,0,0])]
        
        # GSD to structured array
        self.GSD = self.GSD_to_array(GSD)
        self.plf, self.Fl, self.Fs = self.GSD_fractions(self.GSD)
        self.sand_fraction = self.sand_fraction()
        
        self.friction = friction

        self.channel_width = float(channel_width)
        self.base_level = float(base_level_elevation)
        self.roughness_factor = float(roughness_factor)
        self.active_layer_factor = float(active_layer_factor)
        self.substrate_coeff = float(substrate_coeff)
        

        self.qf = float(Q_flood)  / self.channel_width
        self.If = float(intermittency)
        self.qg = float(Q_gravel) / self.channel_width

        self.porosity = float(porosity)
        self.S = float(bed_slope)
        self.L = float(reach_length)
        self.dx = float(dx)
        
        # Constants
        self.g = float(g)
        self.sec_to_year = 60 * 60 * 24 * 365.25
        self.day_to_sec = 60 * 60 * 24

        self.dt = float(dt) * self.day_to_sec # input in days, change to secs
        self.totalT = float(totalT)

        self.alpha_u = float(upwind_coeff) # upwind coefficient [1=full upwind, 0.5=central difference]
        self.alpha_r = float(manning_coeff) # coefficient in Manning resistance relation

        # Chezy
        self.Cf = float(chezy_friction_coeff)

        # Derivatives
        self._R = (float(rho_s) - float(rho_w)) / float(rho_w)


        # arrays
        self.M = int(self.L/self.dx) # number of sub-reaches
        
        self.x = np.array(range(self.M+1)) * self.dx
        self.eta = self.S*self.L - self.S*self.x
        self.eta[-1] = 0

        self.qb = np.zeros((self.M+1,), dtype = np.float)
        self.Sl = np.zeros_like(self.qb)
        self.tau = np.zeros_like(self.qb)
        self.Ht = np.zeros_like(self.qb)
        
        
        # constants of unknown use
        tausc50 = 0.03, tausr50 = 0.04, aload = 11.2, nload = 4.5
        rload = tausr50/tausc50
        if rload>1:
            Cexp = aload * (1 - (1/rload))**nload
            nexp = nload / (rload * (1 - (1/rload)))
        else:
            Cexp = 0
            nexp = 0
        

    def GSD_to_array(self, GSD):
        
        dtype = [('di', 'f4'), ('plff', 'f4'), ('Ffl', 'f4'), ('Ffs', 'f4')]

        GSD_vals = np.zeros((len(GSD)), dtype = dtype)

        for j in range(3):

            GSD_sub = [(GSD[i][0],GSD[i][1][0]) for i in range(len(GSD))]
            datapts = gsd_calc.pre_process_data(GSD_sub)

            GSD_vals['di'] = datapts['diameters']
            GSD_vals[GSD_vals.dtype.names[j+1]] = datapts['fractions'] * 100
            
        return GSD_vals
    
    
    def GSD_fractions(self, GSD):

        plf = np.diff(GSD['plff'])
        Fl = np.diff(GSD['Ffl'])
        Fs = np.diff(GSD['Ffs'])
        
        return plf, Fl, Fs
        
        

In [7]:
GSD = [(256,[100,100,100]),
                    (128,[95,95,95]),
                    (64,[80,80,80]),
                    (32,[50,50,50]),
                    (16,[25,25,25]),
                    (8,[10,10,10]),
                    (4,[5,5,5]),
                    (2,[0,0,0]),
                    (1,[0,0,0]),
                    (0.5,[0,0,0]),
                    (0.25,[0,0,0]),
                    (0.125,[0,0,0])]

<module 'GSD_calculator' from 'GSD_calculator.py'>

In [51]:
dtype = [('di', 'f4'), ('plff', 'f4'), ('Ffl', 'f4'), ('Ffs', 'f4')]

GSD_vals = np.zeros((len(GSD)), dtype = dtype)

for j in range(3):

    GSD_sub = [(GSD[i][0],GSD[i][1][0]) for i in range(len(GSD))]
    datapts = gsd_calc.pre_process_data(GSD_sub)
    
    GSD_vals['di'] = datapts['diameters']
    GSD_vals[GSD_vals.dtype.names[j+1]] = datapts['fractions'] * 100
    
plf = np.diff(GSD_vals['plff'])
Fl = np.diff(GSD_vals['Ffl'])
Fs = np.diff(GSD_vals['Ffs'])

In [60]:
np.diff(GSD_vals[GSD_vals.dtype.names[1]])

array([  0.,   0.,   0.,   0.,   5.,   5.,  15.,  25.,  30.,  15.,   5.], dtype=float32)

In [57]:
GSD_vals['plff']

array([   0.,    0.,    0.,    0.,    0.,    5.,   10.,   25.,   50.,
         80.,   95.,  100.], dtype=float32)